In [1]:
from ibl_pipeline.analyses import behavior
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, action, acquisition
from ibl_pipeline.utils import psychofit as psy
from uuid import UUID
import plotting_utils as putils
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
from plotly import tools
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

Connecting shan@datajoint.internationalbrainlab.org:3306


In [2]:
subj = subject.Subject & 'subject_nickname="CSHL_007"'
# get fit parameters by date
fit_pars = (behavior.BehavioralSummaryByDate.PsychResults * subj).proj(
    'session_date', 'prob_left','threshold', 'bias', 'lapse_low', 'lapse_high').fetch(as_dict=True)
fit_pars = pd.DataFrame(fit_pars)
par_names = ['threshold', 'bias', 'lapse_low', 'lapse_high']

d = putils.get_date_range(subj)
status = putils.get_status(subj)

thresholds = [[19, 19], [16, 16, -16, -16], [0.2, 0.2], [0.2, 0.2]]
xranges = [[d['first_date_str'], d['last_date_str']],
           [d['first_date_str'], d['last_date_str'], d['last_date_str'], d['first_date_str']],
           [d['first_date_str'], d['last_date_str']],
           [d['first_date_str'], d['last_date_str']]
          ]
yranges = [[0, 100], [-100, 100], [0, 1], [0, 1]]

In [3]:
pars = dict()
for par_name in par_names:
    pars[par_name] = []
    
prob_lefts = fit_pars['prob_left'].unique()
    
for iprob_left, prob_left in enumerate(prob_lefts):
    prob_left_filter = fit_pars['prob_left']==prob_left   
    dot_color, error_color=putils.get_color(prob_left)
    
    fit_pars_sub = fit_pars[prob_left_filter]
    
    for ipar, par_name in enumerate(par_names):
        if ipar==0:
            show_legend = True
        else:
            show_legend = False
        pars[par_name].append(
            go.Scatter(      
                x=[t.strftime('%Y-%m-%d') for t in fit_pars_sub['session_date'].tolist()],
                y=fit_pars_sub[par_name].tolist(),
                mode='markers',
                marker=dict(
                    size=5,
                    color=dot_color,
                    opacity=0.8
                ),
                name=f'p_left = {prob_left}',
                xaxis='x{}'.format(4-ipar),
                yaxis='y{}'.format(4-ipar),
                showlegend=show_legend,
                legendgroup='p_left'
            ))

In [4]:
pars_data = [pars[par_name][i] for i, prob_left in enumerate(prob_lefts)
             for par_name in par_names]


for ipar, par_name in enumerate(par_names): 
    if ipar==0:
        show_legend = True
    else:
        show_legend = False
    
    pars_data.append(
        go.Scatter(
            x=xranges[ipar],
            y=thresholds[ipar],
            mode="lines",
            line=dict(
                width=1,
                color='darkgreen',
                dash='dashdot'),
            name='threshold for trained',
            xaxis='x{}'.format(4-ipar),
            yaxis='y{}'.format(4-ipar),
            showlegend=show_legend,
            legendgroup='date'
        )
    )
    
    # add monday plots
    pars_data = putils.create_monday_plot(
        pars_data, yranges[ipar], d['mondays'], 
        xaxis='x{}'.format(4-ipar),
        yaxis='y{}'.format(4-ipar),
        show_legend_external=show_legend
    )

    # add status plots
    pars_data = putils.create_status_plot(
        pars_data, yranges[ipar], status, 
        xaxis='x{}'.format(4-ipar),
        yaxis='y{}'.format(4-ipar),
        show_legend_external=show_legend
    )

x_axis_range = [d['first_date_str'], (d['last_date'] - datetime.timedelta(days=1)).strftime('%Y-%m-%d')]
layout = go.Layout(
    xaxis1=dict(
        domain=[0, 1],
        range=x_axis_range,
        title='Date',
        showgrid=False
    ),
    yaxis1=dict(
        domain=[0, 0.2],
        anchor='x1',
        showgrid=False,
        range=[-0.02, 1.02],
        title='$Lapse high\ (\\lambda)$'
    ),
    xaxis2=dict(
        domain=[0, 1],
        range=x_axis_range,
        showgrid=False
    ),
    yaxis2=dict(
        domain=[0.25, 0.45],
        anchor='x2',
        showgrid=False,
        range=[-0.02, 1.02],
        title='$Lapse low\ (\\gamma)$'
    ),
    xaxis3=dict(
        domain=[0, 1],
        range=x_axis_range,
        showgrid=False
    ),
    yaxis3=dict(
        domain=[0.5, 0.7],
        anchor='x3',
        showgrid=False,
        range=[-105, 105],
        title='$Bias\ (\\mu)$'
    ),
    xaxis4=dict(
        domain=[0, 1],
        range=x_axis_range,
        showgrid=False
    ),
    yaxis4=dict(
        domain=[0.75, 1],
        anchor='x4',
        showgrid=False,
        range=[-5, 105],
        title='$Threshold\ (\\sigma)$'
    ),
    height=1000, 
    width=600,
    title=dict(
        text='Fit Parameters',
        x=0.3,
        y=0.93
    ),
    legend=dict(
        x=1.1,
        y=1,
        orientation='v')
)

fig = go.Figure(data=pars_data, layout=layout)

plotly.offline.iplot(fig)

In [5]:
f = open('fit_pars.json', 'w')
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()